In [1]:
%cd D:\이지메디컴

d:\이지메디컴


In [7]:
import pandas as pd
import sys
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.common.exceptions import NoSuchElementException, TimeoutException,StaleElementReferenceException
import urllib.request
import requests
from datetime import datetime
import time as time
from selenium.common.exceptions import UnexpectedAlertPresentException

In [8]:
# options = webdriver.ChromeOptions()
# options.add_argument('headless') #headless 모드로 동작하도록 만들어주는 키워드
# options.add_argument('disable-gpu') #빠른 화면 렌더링을 실행하게 되면 크롬에서 버그를 일으킬 수도 있음!

driver = webdriver.Chrome('C:/webdrivers/chromedriver.exe')
driver.implicitly_wait(3)
driver.get('https://udiportal.mfds.go.kr/search/data/P02_01')
driver.implicitly_wait(1)

#options 테스트용
# driver.get('http://naver.com')
# driver.implicitly_wait(3)
# driver.get_screenshot_as_file('naver_main_headless.png')
# driver.quit()

In [9]:
def getKeys(수허):
    driver.implicitly_wait(3)

    #단어 설정
    searchForm = driver.find_element_by_xpath('//*[@id="itemNoFullname"]')
    searchForm.send_keys(수허)
    driver.implicitly_wait(3)

    #등급 설정
    grade = driver.find_element_by_xpath('//*[@id="grade"]/option[4]')
    grade.click()
    driver.implicitly_wait(3)

    #검색버튼 클릭
    driver.find_element_by_xpath('//*[@id="searchBtn"]').click()

    #한 페이지 출력 개수 설정
    driver.find_element_by_xpath('//*[@id="recordCountPerPage"]/option[4]').click()
    driver.implicitly_wait(3)

    #go 버튼 클릭
    driver.find_element_by_xpath('//*[@id="searchResultList"]/div/div[1]/div/fieldset/button').click()

In [10]:
getKeys('수허')

In [15]:
# count = 0
# page = 3
# driver.find_element_by_xpath('//*[@id="searchResultList"]/div/div[3]/ul/li[7]/a').click()
# driver.implicitly_wait(3)
# # driver.find_element_by_xpath('//*[@id="searchResultList"]/div/div[3]/ul/li[7]/a').click()
# # driver.implicitly_wait(3)
# # driver.find_element_by_xpath('//*[@id="searchResultList"]/div/div[3]/ul/li[7]/a').click()


In [6]:
def crawller():

    dic = {"업체명" : [] , "품목명" : [] , "모델명":[], "품목허가번호":[] ,"등급" : [] ,"요양급여대상" : [] ,"요양급여코드" : [],"요양 코드 미입력 사유" : []}
    df1 = pd.DataFrame(dic)    

    count = 0
    page = 3
    
    
    
    
    while True:

        try:
        # 게시판 !!!

            time.sleep(3)

#             company_name = driver.find_element_by_id('item_'+str(count)).text

#             item_name = driver.find_element_by_xpath('//*[@id="itemTr_'+str(count)+'"]/td[2]').text

#             model_name = driver.find_element_by_xpath('//*[@id="itemTr_'+str(count)+'"]/td[3]').text

#             assigned_number = driver.find_element_by_xpath('//*[@id="itemTr_'+str(count)+'"]/td[5]').text


            driver.find_element_by_id('item_'+str(count)).send_keys("\n") #아이템을 눌러주는 기능
            time.sleep(3)




            itemName = driver.find_element_by_xpath('//*[@id="mg_itemName"]').text #품목명
            grade = driver.find_element_by_xpath('//*[@id="mg_grade"]').text #등급
            modelName= driver.find_element_by_xpath('//*[@id="mg_mebTypeName"]').text #모델명
            meddiv_itemNo =  driver.find_element_by_xpath('//*[@id="mg_meddevItemNo"]').text #품목허가번호
            entpName =  driver.find_element_by_xpath('//*[@id="pg_entpName"]').text #상호명
            pg_rcper_target =  driver.find_element_by_xpath('//*[@id="pg_rcperSalaryTargetYn"]').text #요양급여대상
            pg_rcper_code =  driver.find_element_by_xpath('//*[@id="pg_rcperSalaryCode"]').text #요양급여코드
            pc_rcper_no_reason =  driver.find_element_by_xpath('//*[@id="pg_rcperSalaryCodeNoinputReas"]').text #요양급여코드미입력사유

            dic = {"업체명" : [entpName] , "품목명" : [itemName] , "모델명":[modelName], "품목허가번호":[meddiv_itemNo] ,"등급" : [grade],
                   "요양급여대상" : [pg_rcper_target] ,"요양급여코드" : [pg_rcper_code],"요양 코드 미입력 사유" : [pc_rcper_no_reason]}





            df = pd.DataFrame(dic)
            print(df)
            df1 = pd.concat([df1,df])

            df1.to_csv("수허_3등급_3차(UTF-8).csv",encoding='UTF-8')


            try:
                driver.find_element_by_xpath('//*[@id="closeBtnBot"]').click()
            except :
                pass



            count = count +1


        #한 페이지에 있는 아이템들을 다 뽑았으면?
            if count > 49:


                try:
                    print("----------------다음 페이지----------------")
                    time.sleep(3)

                    #다음 페이지로 넘어가게 해주는 기능 (2부터 7까지 있음, 2는 맨 처음 페이지, 7은 '다음')
                    #'다음'을 누르면 첫 페이지는 디폴트기 때문에, 넘어갈 때 2페이지로 갈 수 있도록 설정해준다.
                    driver.find_element_by_xpath('//*[@id="searchResultList"]/div/div[3]/ul/li['+str(page)+']/a').click() 


                    if page >= 7:
                        count = 0 #페이지가 넘어가면 아이템 0번부터 다시 크롤링
                        page = 3
                        time.sleep(3)


                    else: #페이지가 7이 넘어가지 않았다면 ('다음'을 누르지 않은 상태라면)
                        count = 0
                        page = page +1 #원래 페이지에서 하나 더 늘어있게!

                        time.sleep(3)
                        print("----------------"+str(page-2)+"----------------")
                        
                        if data.responseJSON.mebItemStatus == 'D':
                            
                            alert("해당 모델정보는 현재 존재하지 않습니다.");
                            goSearch('136');

                except NoSuchElementException:


                    break
                #여기서 보내기






        except NoSuchElementException as e:
            print(df1)

            print(e)

            break
        except UnexpectedAlertPresentException:
            break
#             driver.find_element_by_xpath('//*[@id="searchResultList"]/div/div[3]/ul/li['+str(page)+']/a').click()
        except:
            continue

    return df1



if __name__ == "__main__":
    
    a = crawller()
    a.to_csv("수허_3등급_3차(UTF-8).csv",encoding='UTF-8')

          업체명      품목명         모델명       품목허가번호 등급 요양급여대상    요양급여코드  \
0  한국스트라이커(주)  골수내고정막대  1843-1526S  수허 01-366 호  3      예  C3113024   

  요양 코드 미입력 사유  
0               
          업체명      품목명         모델명       품목허가번호 등급 요양급여대상    요양급여코드  \
0  한국스트라이커(주)  골수내고정막대  1843-1528S  수허 01-366 호  3      예  C3113024   

  요양 코드 미입력 사유  
0               
          업체명      품목명         모델명       품목허가번호 등급 요양급여대상    요양급여코드  \
0  한국스트라이커(주)  골수내고정막대  1843-1530S  수허 01-366 호  3      예  C3113024   

  요양 코드 미입력 사유  
0               
          업체명      품목명         모델명       품목허가번호 등급 요양급여대상    요양급여코드  \
0  한국스트라이커(주)  골수내고정막대  1843-1532S  수허 01-366 호  3      예  C3113024   

  요양 코드 미입력 사유  
0               
             업체명      품목명          모델명        품목허가번호 등급 요양급여대상 요양급여코드  \
0  (유)짐머바이오메트코리아  골수내고정막대  9032-08-000  수허 03-1106 호  3                 

  요양 코드 미입력 사유  
0               
             업체명      품목명          모델명        품목허가번호 등급 요양급여대상 요양급여코드  \
0  (유)짐머바이오메트코리아  골수내고정막대  1812-10-420

### 201페이지부터 돌려야 하는 코드

In [15]:
def crawller():

    dic = {"업체명" : [] , "품목명" : [] , "모델명":[], "품목허가번호":[] ,"등급" : [] ,"요양급여대상" : [] ,"요양급여코드" : [],"요양 코드 미입력 사유" : []}
    df1 = pd.DataFrame(dic)    

    count = 0
    page = 2
    while True:

        try:
        # 게시판 !!!

            time.sleep(3)

#             company_name = driver.find_element_by_id('item_'+str(count)).text

#             item_name = driver.find_element_by_xpath('//*[@id="itemTr_'+str(count)+'"]/td[2]').text

#             model_name = driver.find_element_by_xpath('//*[@id="itemTr_'+str(count)+'"]/td[3]').text

#             assigned_number = driver.find_element_by_xpath('//*[@id="itemTr_'+str(count)+'"]/td[5]').text


            driver.find_element_by_id('item_'+str(count)).send_keys("\n")
            time.sleep(3)




            itemName = driver.find_element_by_xpath('//*[@id="mg_itemName"]').text #품목명
            grade = driver.find_element_by_xpath('//*[@id="mg_grade"]').text #등급
            modelName= driver.find_element_by_xpath('//*[@id="mg_mebTypeName"]').text #모델명
            meddiv_itemNo =  driver.find_element_by_xpath('//*[@id="mg_meddevItemNo"]').text #품목허가번호
            entpName =  driver.find_element_by_xpath('//*[@id="pg_entpName"]').text #상호명
            pg_rcper_target =  driver.find_element_by_xpath('//*[@id="pg_rcperSalaryTargetYn"]').text #요양급여대상
            pg_rcper_code =  driver.find_element_by_xpath('//*[@id="pg_rcperSalaryCode"]').text #요양급여코드
            pc_rcper_no_reason =  driver.find_element_by_xpath('//*[@id="pg_rcperSalaryCodeNoinputReas"]').text #요양급여코드미입력사유

            dic = {"업체명" : [entpName] , "품목명" : [itemName] , "모델명":[modelName], "품목허가번호":[meddiv_itemNo] ,"등급" : [grade],
                   "요양급여대상" : [pg_rcper_target] ,"요양급여코드" : [pg_rcper_code],"요양 코드 미입력 사유" : [pc_rcper_no_reason]}





            df = pd.DataFrame(dic)
            print(df)
            df1 = pd.concat([df1,df])

            df1.to_csv("수허_3등급(UTF-8).csv",encoding='UTF-8')


            try:
                driver.find_element_by_xpath('//*[@id="closeBtnBot"]').click()
            except :
                pass



            count = count +1


        #다 뽑았으면?
            if count > 49:


                try:
                    print("----------------다음 페이지 ----------------------")
                    time.sleep(3)


                    driver.find_element_by_xpath('//*[@id="searchResultList"]/div/div[3]/ul/li['+str(page)+']/a').click()

# //*[@id="searchResultList"]/div/div[3]/ul/li[2]/a #201
# //*[@id="searchResultList"]/div/div[3]/ul/li[3]/a #다음
# //*[@id="searchResultList"]/div/div[3]/ul/li[2]/a #202
# //*[@id="searchResultList"]/div/div[3]/ul/li[3]/a #다음
# //*[@id="searchResultList"]/div/div[3]/ul/li[2]/a
# //*[@id="searchResultList"]/div/div[3]/ul/li[3]/a


                    if page >= 2:
                        count = 0
                        page = 3
                        time.sleep(3)


#                     else:
#                         count = 0
#                         page = page +1

                        time.sleep(3)
                        print("-----------------------------"+str(page-1)+"-----------------------------")

                except NoSuchElementException:


                    break
                #여기서 보내기






        except NoSuchElementException as e:
            print(df1)

            print(e)

            break
        except UnexpectedAlertPresentException:
            break
        except:
            break 
            
    return df1



if __name__ == "__main__":
    
    a = crawller()
    a.to_csv("수허_3등급(UTF-8).csv",encoding='UTF-8')

              업체명      품목명          모델명       품목허가번호 등급 요양급여대상    요양급여코드  \
0  한국존슨앤드존슨메디칼(주)  골수내고정막대  04.009.768S  수허 09-396 호  3      예  C3113103   

  요양 코드 미입력 사유  
0               
              업체명      품목명          모델명       품목허가번호 등급 요양급여대상    요양급여코드  \
0  한국존슨앤드존슨메디칼(주)  골수내고정막대  04.009.256S  수허 09-396 호  3      예  C3113103   

  요양 코드 미입력 사유  
0               
              업체명      품목명          모델명       품목허가번호 등급 요양급여대상 요양급여코드  \
0  한국존슨앤드존슨메디칼(주)  골수내고정막대  04.009.769S  수허 09-396 호  3                 

  요양 코드 미입력 사유  
0               
              업체명      품목명          모델명       품목허가번호 등급 요양급여대상 요양급여코드  \
0  한국존슨앤드존슨메디칼(주)  골수내고정막대  04.009.772S  수허 09-396 호  3                 

  요양 코드 미입력 사유  
0               
              업체명      품목명          모델명       품목허가번호 등급 요양급여대상 요양급여코드  \
0  한국존슨앤드존슨메디칼(주)  골수내고정막대  04.009.773S  수허 09-396 호  3                 

  요양 코드 미입력 사유  
0               
              업체명      품목명          모델명       품목허가번호 등급 요양급여대상    요양급여코드  \
0  한국존슨

### 잘 뽑혔나 체크

In [ ]:
%cd D:\이지메디컴

In [ ]:
%ls

In [ ]:
df = pd.read_csv('수인_3등급(UTF-8).csv',encoding='UTF-8')
df